## Vanilla Policy Optimisation

<a href="https://colab.research.google.com/github/EffiSciencesResearch/ML4G/blob/main/days/w1d5/vanilla_policy_gradient.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Preliminary questions:
- Run the script with the defaults parameters on the terminal
- Explain from torch.distributions.categorical import Categorical
- google gym python, why is it useful?
- Policy gradient is model based or model free?
- Is policy gradient on-policy or off-policy?

Read all the code, then:
- Use https://github.com/patrick-kidger/torchtyping to type the functions get_policy, get_action and compute_loss
- Type completely the whole code.
- Use from typeguard import typechecked and the @typechecked decorator to check the previous question.
- Answer the questions


In [32]:
import torch
import torch.nn as nn
from torch.distributions.categorical import Categorical
from torch.optim import Adam
import numpy as np
import gym
from gym.spaces import Discrete, Box
from torchtyping import TensorType, patch_typeguard
from typeguard import typechecked

patch_typeguard()  # use before @typechecked

def mlp(sizes, activation=nn.Tanh, output_activation=nn.Identity):
    # Build a feedforward neural network.
    layers = []
    for j in range(len(sizes)-1):
        act = activation if j < len(sizes)-2 else output_activation
        layers += [nn.Linear(sizes[j], sizes[j+1]), act()]
    # What does * mean here? Search for unpacking in python
    # We unpack the tuple
    return nn.Sequential(*layers)

def train(env_name='CartPole-v0', hidden_sizes=[32], lr=1e-2, 
          epochs=50, batch_size=5000, render=False):

    # make environment, check spaces, get obs / act dims
    env = gym.make(env_name)
    assert isinstance(env.observation_space, Box), \
        "This example only works for envs with continuous state spaces."
    assert isinstance(env.action_space, Discrete), \
        "This example only works for envs with discrete action spaces."

    obs_dim = env.observation_space.shape[0]
    n_acts = env.action_space.n

    # Core of policy network
    # What should be the sizes of the layers of the policy network?
    logits_net = mlp(sizes=[obs_dim]+hidden_sizes+[n_acts])
    # obs_dim -> ... -> A


    # make function to compute action distribution
    # What is the shape of obs?
    @typechecked
    def get_policy(obs : TensorType[..., obs_dim]) -> Categorical:
        # Warning: obs has not always the same shape.
        logits = logits_net(obs)
        return Categorical(logits=logits)

    # make action selection function (outputs int actions, sampled from policy)
    # What is the shape of obs?
    @typechecked
    def get_action(obs : TensorType[..., obs_dim]) -> int:
        return get_policy(obs).sample().item()

    # make loss function whose gradient, for the right data, is policy gradient
    # What does the weights parameter represents here?
    # What is the shape of obs?
    @typechecked
    def compute_loss(obs : TensorType[..., obs_dim], act : TensorType[...], weights : TensorType[...]) \
      -> TensorType[()]:
        logp = get_policy(obs).log_prob(act)
        return -(logp * weights).mean()

    # make optimizer
    optimizer = Adam(logits_net.parameters(), lr=lr)

    # for training policy
    @typechecked
    def train_one_epoch() -> (TensorType[()], TensorType[...], TensorType[...]):
        # make some empty lists for logging.
        batch_obs = []          # for observations
        batch_acts = []         # for actions
        batch_weights = []      # for R(tau) weighting in policy gradient
        batch_rets = []         # for measuring episode returns # What is the return?
        batch_lens = []         # for measuring episode lengths

        # reset episode-specific variables
        obs = env.reset()       # first obs comes from starting distribution 
        done = False            # signal from environment that episode is over
        ep_rews = []            # list for rewards accrued throughout ep

        # render first episode of each epoch
        finished_rendering_this_epoch = False

        # collect experience by acting in the environment with current policy
        while True:

            # rendering
            if (not finished_rendering_this_epoch) and render:
                env.render()

            # save obs
            batch_obs.append(obs.copy())

            # act in the environment
            act = get_action(torch.as_tensor(obs, dtype=torch.float32))
            obs, rew, done, _ = env.step(act)

            # save action, reward
            batch_acts.append(act)
            ep_rews.append(rew)

            if done:
                # if episode is over, record info about episode
                # Is the reward discounted?
                ep_ret, ep_len = sum(ep_rews), len(ep_rews)
                batch_rets.append(ep_ret)
                batch_lens.append(ep_len)

                # the weight for each logprob(a|s) is R(tau)
                # Why do we use a constant vector here?
                batch_weights += [ep_ret] * ep_len

                # reset episode-specific variables
                obs, done, ep_rews = env.reset(), False, []

                # won't render again this epoch
                finished_rendering_this_epoch = True

                # end experience loop if we have enough of it
                if len(batch_obs) > batch_size:
                    break

        # take a single policy gradient update step
        optimizer.zero_grad()
        batch_loss = compute_loss(obs=torch.as_tensor(batch_obs, dtype=torch.float32),
                                  act=torch.as_tensor(batch_acts, dtype=torch.int32),
                                  weights=torch.as_tensor(batch_weights, dtype=torch.float32)
                                  )
        batch_loss.backward()
        optimizer.step()
        return batch_loss, batch_rets, batch_lens

    # training loop
    for i in range(epochs):
        batch_loss, batch_rets, batch_lens = train_one_epoch()
        print('epoch: %3d \t loss: %.3f \t return: %.3f \t ep_len: %.3f'%
                (i, batch_loss, np.mean(batch_rets), np.mean(batch_lens)))


In [34]:
train(env_name='CartPole-v0', hidden_sizes=[32], lr=1e-2, 
          epochs=50, batch_size=50, render=False)

/usr/local/lib/python3.7/dist-packages/gym/envs/registration.py:594: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  f"The environment {id} is out of date. You should consider "
/usr/local/lib/python3.7/dist-packages/gym/core.py:318: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."
/usr/local/lib/python3.7/dist-packages/gym/wrappers/step_api_compatibility.py:40: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in futu

epoch:   0 	 loss: 19.801 	 return: 26.000 	 ep_len: 26.000
epoch:   1 	 loss: 19.628 	 return: 28.000 	 ep_len: 28.000
epoch:   2 	 loss: 24.686 	 return: 32.000 	 ep_len: 32.000
epoch:   3 	 loss: 28.666 	 return: 27.667 	 ep_len: 27.667
epoch:   4 	 loss: 10.457 	 return: 15.500 	 ep_len: 15.500
epoch:   5 	 loss: 14.449 	 return: 16.750 	 ep_len: 16.750
epoch:   6 	 loss: 27.305 	 return: 33.000 	 ep_len: 33.000
epoch:   7 	 loss: 9.304 	 return: 14.250 	 ep_len: 14.250
epoch:   8 	 loss: 22.223 	 return: 32.500 	 ep_len: 32.500
epoch:   9 	 loss: 18.798 	 return: 22.667 	 ep_len: 22.667
epoch:  10 	 loss: 17.482 	 return: 20.500 	 ep_len: 20.500
epoch:  11 	 loss: 10.588 	 return: 14.750 	 ep_len: 14.750
epoch:  12 	 loss: 12.180 	 return: 18.333 	 ep_len: 18.333
epoch:  13 	 loss: 11.815 	 return: 16.250 	 ep_len: 16.250
epoch:  14 	 loss: 24.219 	 return: 27.500 	 ep_len: 27.500
epoch:  15 	 loss: 12.327 	 return: 16.750 	 ep_len: 16.750
epoch:  16 	 loss: 8.695 	 return: 13.000

In [ ]:
# Original algo here: https://github.com/openai/spinningup/blob/master/spinup/algos/pytorch/vpg/vpg.py